In [ ]:
import json

import ipycytoscape
import networkx as nx
import rdflib
from rdflib.namespace import RDFS

from ipyradiant import FileManager, InteractiveViewer, PathLoader
from ipyradiant.rdf2nx import RDF2NX

## Set up graph using base from `RDF_to_NX.ipynb`

### We are manually populating an initial cytoscape graph that will be used in this example. We run a query on the Star Wars API in order to get a few basic starting nodes that we will use as a base point in graph exploration.

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
qres = lw.graph.query(
    """
    PREFIX hum: <https://swapi.co/resource/human/>
    PREFIX film: <https://swapi.co/resource/film/>
    
    CONSTRUCT {
        ?s ?p ?o .
    }
    WHERE {
        ?s ?p ?o .
        
        VALUES (?s) {
            (hum:1)  # Luke
            (hum:4)  # Vader
            (film:1) # A New Hope
        }
    }
    """
)

simple_graph = rdflib.graph.Graph().parse(data=qres.serialize(format="xml"))
uri = RDFS.label
ns = {"rdfs": str(RDFS)}
initNs = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "res": "https://swapi.co/resource/",
    "voc": "https://swapi.co/vocabulary/",
    "base": "https://swapi.co/resource/",
}

nx_graph = RDF2NX.convert(simple_graph, namespaces=initNs)

Create a basic cytoscape graph.

In [ ]:
directed = ipycytoscape.CytoscapeWidget()
directed.graph.add_graph_from_networkx(nx_graph, multiple_edges=True, directed=True)
for node in directed.graph.nodes:
    # deal with inability to handle colons
    node.data["_label"] = node.data.get("rdfs:label", None)
    node.data["_attrs"] = json.dumps(node.data, indent=2)

### Set up the `InteractiveViewer` by passing it an `rdf_graph` for queries and also a `cytoscape_widget` which is the starting `ipycytoscape.CytoscapeWidget()` object.

In [ ]:
iv = InteractiveViewer()
iv.rdf_graph = rdf_graph
iv.cytoscape_widget = directed

### Display the `InteractiveViewer` object and notice how you can traverse the graph by adding nodes, **double clicking** to make nodes permanent, and then removing non-permanent (temporary) nodes.

In [ ]:
iv